In [78]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tslearn.metrics import dtw, soft_dtw, dtw_path
from sklearn.metrics import accuracy_score
from scipy.spatial import distance as dist
from scipy.stats import norm

from reader import get_x_label, get_class_x
from average import amplitude_avg, dtw_avg, soft_dtw_avg

In [197]:
file_name = '50words'

In [198]:
class_x = get_class_x(file_name + '_TRAIN')
class_avg = dtw_avg(class_x)

In [199]:
dtw_stat = [[list() for i in range(len(class_x[0][0]))] for _ in range(len(class_x))]
for i in range(class_x.shape[0]):
    for j in range(len(class_x[i])):
        path, dist = dtw_path(class_avg[i], class_x[i][j])
        for avg_pos, x_pos in path:
            dtw_stat[i][avg_pos].append(class_x[i][j][x_pos])
dtw_stat = np.array(dtw_stat)
for i in range(dtw_stat.shape[0]):
    for j in range(dtw_stat.shape[1]):
        dtw_stat[i] = np.array(dtw_stat[i])

In [200]:
dtw_mean = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
dtw_sd = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
for class_label in range(dtw_stat.shape[0]):
    for point in range(dtw_stat.shape[1]):
        stat = np.array(dtw_stat[class_label][point])
        dtw_mean[class_label][point].append(stat.mean())
        dtw_sd[class_label][point].append(stat.std())

In [201]:
test_x, test_label = get_x_label(file_name + '_TEST')

In [204]:
bay_predict = []
dist_predict = []
EPSILON = 10**(-10)
count = 0
for x in test_x:
    min_log_prob = 10**(310)
    min_dist = 100000
    bay_nearest_class = -1
    dist_nearest_class = -1
    for j in range(len(class_avg)):
        # Class j
        path, dist = dtw_path(x, class_avg[j])
        log_prob = 1
        probs = [list() for _ in range(len(x))]
        for x_pos, avg_pos in path:
            area_up = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).cdf(x[x_pos])
            area_low = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).cdf(x[x_pos] - EPSILON)
            prob = area_up - area_low
            if prob < 10**(-310):
                prob = 10**(-310)
            probs[x_pos].append(prob)
        for pos_probs in probs:
#             print(len(pos_probs))
            max_pos_prob = max(pos_probs)
#             print(max_pos_prob)
            log_prob += -np.log(max_pos_prob)
#         print(log_prob)
        if log_prob < min_log_prob:
            min_log_prob = log_prob
            bay_nearest_class = j
        if dist < min_dist:
            min_dist = dist
            dist_nearest_class = j
    bay_predict.append(bay_nearest_class)
    dist_predict.append(dist_nearest_class)
    print(count, ':', bay_nearest_class, dist_nearest_class, test_label[count]-1)
    count += 1

/Users/sirinthra-cc/.pyenv/versions/3.6.2/envs/ml/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1735: RuntimeWarning: divide by zero encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)


0 : 36 3 3
1 : 38 4 11
2 : 13 13 12
3 : 31 22 22
4 : 36 3 3
5 : 12 13 12
6 : 31 26 26
7 : 0 0 0
8 : 21 21 21
9 : 0 0 0
10 : 18 39 6
11 : 7 37 8
12 : 6 42 42
13 : 13 13 12
14 : 17 7 7
15 : 7 39 2
16 : 8 8 1
17 : 5 5 5
18 : 27 23 23
19 : 13 8 8
20 : 5 5 5
21 : 0 47 0
22 : 14 14 14
23 : 31 23 2
24 : 42 7 7
25 : 23 32 10
26 : 7 0 15
27 : 21 9 9
28 : 4 1 1
29 : 4 8 1
30 : 4 4 4
31 : 8 1 1
32 : 2 2 2
33 : 10 3 3
34 : 15 0 0
35 : 15 0 0
36 : 5 5 5
37 : 22 32 2
38 : 43 0 0
39 : 4 1 1
40 : 10 32 32
41 : 0 0 0
42 : 5 5 5
43 : 8 8 8
44 : 3 3 3
45 : 22 21 41
46 : 13 29 29
47 : 12 13 13
48 : 28 28 28
49 : 15 30 47
50 : 15 29 39
51 : 27 32 40
52 : 7 7 7
53 : 29 13 13
54 : 28 13 44
55 : 45 45 11
56 : 15 15 15
57 : 15 0 0
58 : 28 7 7
59 : 6 6 6
60 : 31 17 33
61 : 5 0 0
62 : 15 0 0
63 : 22 2 2
64 : 28 28 28
65 : 2 25 25
66 : 12 30 30
67 : 4 1 1
68 : 2 27 5
69 : 15 0 0
70 : 5 0 0
71 : 4 8 4
72 : 31 41 41
73 : 15 0 0
74 : 43 4 4
75 : 22 21 21
76 : 22 23 23
77 : 7 0 0
78 : 27 27 27
79 : 0 16 16
80 : 2 32 

In [205]:
accuracy_score(test_label-1, bay_predict)

0.33406593406593404